#### Imports

In [10]:
import grpc
import tensorflow as tf
from keras_image_helper import create_preprocessor
from tensorflow.keras.models import load_model
from tensorflow_serving.apis import prediction_service_pb2_grpc, predict_pb2

In [11]:
import os

if os.environ.get('https_proxy'):
 del os.environ['https_proxy']
if os.environ.get('http_proxy'):
 del os.environ['http_proxy']

In [12]:
host = 'localhost:8500'

# channel = grpc.insecure_channel(host)
channel = grpc.insecure_channel(host, options=(('grpc.enable_http_proxy', 0),))

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [13]:
preprocessor = create_preprocessor('xception', target_size=(32, 32))

In [14]:
url = 'https://c7.alamy.com/comp/K0W4HC/a-traffic-sign-indicating-a-speed-limit-of-50-kmh-seen-near-tubingen-K0W4HC.jpg'
X = preprocessor.from_url(url)

In [15]:
# X

In [16]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [17]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'traffic-sign-recognition-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [18]:
# pb_request

In [19]:
pb_response = stub.Predict(pb_request, timeout=20.0)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8500: Failed to connect to remote host: Connection refused"
	debug_error_string = "UNKNOWN:failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8500: Failed to connect to remote host: Connection refused {grpc_status:14, created_time:"2025-01-13T18:14:37.798083-05:00"}"
>

In [20]:
pb_response

NameError: name 'pb_response' is not defined

In [21]:
preds = pb_response.outputs['dense_7'].float_val

NameError: name 'pb_response' is not defined

In [22]:
classes = [
    'Speed limit (20km/h)',
    'Speed limit (30km/h)',
    'Speed limit (50km/h)',
    'Speed limit (60km/h)',
    'Speed limit (70km/h)',
    'Speed limit (80km/h)',
    'End of speed limit (80km/h)',
    'Speed limit (100km/h)',
    'Speed limit (120km/h)',
    'No passing',
    'No passing for vehicles over 3.5 metric tons',
    'Right-of-way at the next intersection',
    'Priority road',
    'Yield',
    'Stop',
    'No vehicles',
    'Vehicles over 3.5 metric tons prohibited',
    'No entry',
    'General caution',
    'Dangerous curve to the left',
    'Dangerous curve to the right',
    'Double curve',
    'Bumpy road',
    'Slippery road',
    'Road narrows on the right',
    'Road work',
    'Traffic signals',
    'Pedestrians',
    'Children crossing',
    'Bicycles crossing',
    'Beware of ice/snow',
    'Wild animals crossing',
    'End of all speed and passing limits',
    'Turn right ahead',
    'Turn left ahead',
    'Ahead only',
    'Go straight or right',
    'Go straight or left',
    'Keep right',
    'Keep left',
    'Roundabout mandatory',
    'End of no passing',
    'End of no passing by vehicles over 3.5 metric'
]

In [23]:
dict(zip(classes, preds))

NameError: name 'preds' is not defined